## Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestRegressor

## Neural Network with 1 hidden layer

In [ ]:
# Simple MLP (Multi-Layer Perceptron)
class ScalingFactorPredictor(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):

    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)  # First layer
    self.relu = nn.ReLU()                          # Activation function (can try different ones)
    self.fc2 = nn.Linear(hidden_size, output_size) # Output layer

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Random Forest

In [ ]:
# Random forest regression model

rf_reg = RandomForestRegressor(n_estimators=100)

## Data Processing Stuff (TBD)